<a href="https://colab.research.google.com/github/bebyakinb/test_assignments_Analyst/blob/master/habr/SQL_Hardest_Interview_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Подготовка среды

In [1]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz &> /dev/null
!tar xzf sqlite.tar.gz &> /dev/null
%cd sqlite/
!./configure &> /dev/null
!make sqlite3.c &> /dev/null
%cd /content
!npx degit coleifer/pysqlite3 -f &> /dev/null
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build &> /dev/null
!cp build/lib.linux-x86_64-3.8/pysqlite3/_sqlite3.cpython-38-x86_64-linux-gnu.so \
    /usr/lib/python3.8/lib-dynload/_sqlite3.cpython-38-x86_64-linux-gnu.so

/content/sqlite
/content


In [1]:
import sqlite3
sqlite3.sqlite_version

'3.40.0'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Задача №1. Процентное изменение месяц к месяцу


## 1.0. Постановка задачи
**Контекст:** часто полезно знать, как изменяется ключевая метрика, например, месячная аудитория активных пользователей, от месяца к месяцу. Допустим у нас есть таблица logins в таком виде:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```
**Задача:** Найти ежемесячное процентное изменение месячной аудитории активных пользователей (MAU).

## 1.1. Подготовка данных

In [3]:
def random_dates(start, end, n=10):
    '''
    Dividing the unix time values(start and end) by 24*60*60*10**9
    to make it days and return n randomly chosen days back.
    '''
    start_u = start.value//(24*60*60*10**9)
    end_u = end.value//(24*60*60*10**9)

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='D')


In [4]:
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2022-12-01')
login_lines = 100000
user_ids = 30000

logins = pd.DataFrame()
logins['user_id'] = np.random.randint(1, user_ids, login_lines)
logins['date'] = pd.Series(random_dates(start, end, login_lines))
logins

,user_id,date
0,12288,2022-11-02
1,15552,2022-10-02
2,19120,2022-09-07
3,27212,2021-09-27
4,15570,2021-09-11
...,...,...
99995,20979,2021-10-16
99996,18852,2021-12-21
99997,16184,2021-09-10
99998,5541,2022-11-30


In [5]:
con = sqlite3.connect('db')
cur = con.cursor()

In [6]:
logins.to_sql('logins', con, if_exists='replace',index=False)

In [7]:
def select(sql):
  return pd.read_sql(sql,con)

In [8]:
sql = '''select * from logins'''
select(sql)

,user_id,date
0,12288,2022-11-02 00:00:00
1,15552,2022-10-02 00:00:00
2,19120,2022-09-07 00:00:00
3,27212,2021-09-27 00:00:00
4,15570,2021-09-11 00:00:00
...,...,...
99995,20979,2021-10-16 00:00:00
99996,18852,2021-12-21 00:00:00
99997,16184,2021-09-10 00:00:00
99998,5541,2022-11-30 00:00:00


## 1.2. Решение SQL

In [9]:
sql ='''
with months_active as (
  select strftime('%Y %m', date) as year_month,
         cast(count(user_id) as real) as users from logins
  group by year_month)

select 
year_month, users, (users - LAG(users) over(order by year_month))*100/LAG(users) over(order by year_month) MAU_percent
from months_active
'''
select(sql)

,year_month,users,MAU_percent
0,2021 01,4415.0,NaN
1,2021 02,4100.0,-7.134768
2,2021 03,4401.0,7.341463
3,2021 04,4347.0,-1.226994
4,2021 05,4455.0,2.484472
5,2021 06,4196.0,-5.813692
6,2021 07,4406.0,5.004766
7,2021 08,4412.0,0.136178
8,2021 09,4342.0,-1.586582
9,2021 10,4350.0,0.184247


## 1.3 Решение Pandas

In [10]:
pt = logins.pivot_table(index=[(logins['date'].dt.to_period('M'))], values=['user_id'], aggfunc="count")
pt['MAU,%mm'] = (pt['user_id'] - pt['user_id'].shift(1))/pt['user_id'].shift(1)*100
pt

,user_id,"MAU,%mm"
date,,
2021-01,4415,NaN
2021-02,4100,-7.134768
2021-03,4401,7.341463
2021-04,4347,-1.226994
2021-05,4455,2.484472
2021-06,4196,-5.813692
2021-07,4406,5.004766
2021-08,4412,0.136178
2021-09,4342,-1.586582


# 2.Задача №2. Маркировка древовидной структуры

## 2.0 Постановка задачи
Контекст: предположим, у вас есть таблица tree с двумя столбцами: в первом указаны узлы, а во втором — родительские узлы.
```
node   parent
1       2
2       5
3       5
4       3
5       NULL 
```
Задача: написать SQL таким образом, чтобы мы обозначили каждый узел как внутренний (inner), корневой (root) или конечный узел/лист (leaf), так что для вышеперечисленных значений получится следующее:
```
node    label  
1       Leaf
2       Inner
3       Inner
4       Leaf
5       Root
```

## 2.1 Подготовка данных

In [11]:
nodes = pd.DataFrame({'node': [1,2,3,4,5],
                      'parent':[2,5,5,3,np.nan]})
nodes

,node,parent
0,1,2.0
1,2,5.0
2,3,5.0
3,4,3.0
4,5,NaN


In [12]:
nodes.to_sql('nodes', con, if_exists='replace',index=False)

### 2.2 Решение SQL

In [13]:
sql = '''
select node,
       case
            when parent is null then 'Root'
            when node in (select parent from nodes) then 'Inner'
            else 'Leaf'
       end as label
from nodes
'''
select(sql)

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


## 2.3 Решение Pandas

In [14]:
nodes['label'] =  np.where(nodes['parent'].isna(), 'Root',
                  np.where(nodes['node'].isin(nodes['parent']), 'Inner', 'Leaf'))
nodes[['node','label']]

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


# 3. Задача № 3. Удержание пользователей в месяц.

## 3.0. Постановка задачи

**Контекст:** допустим, у нас есть статистика по авторизации пользователей на сайте в таблице logins:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```

**Задача:** написать запрос, который получает количество удержанных пользователей в месяц. В нашем случае данный параметр определяется как количество пользователей, которые авторизовались в системе и в этом, и в предыдущем месяце.


## 3.1. Подготовка данных

In [15]:
logins.to_sql('logins', con, if_exists='replace',index=False)

## 3.2. Решение SQL

In [16]:
sql = '''
with months_logins as (
  select distinct user_id, 
         strftime('%Y %m', date) as cur_month,
         strftime('%Y %m', date(date, 'start of month','-1 month')) as prev_month
  from logins
  )

select m1.cur_month, count(m2.user_id)
from months_logins as m1
left join months_logins as m2 ON m1.user_id = m2.user_id AND m1.prev_month = m2.cur_month
group by m1.cur_month
'''
select(sql)

,cur_month,count(m2.user_id)
0,2021 01,0
1,2021 02,484
2,2021 03,534
3,2021 04,572
4,2021 05,544
5,2021 06,562
6,2021 07,547
7,2021 08,561
8,2021 09,553
9,2021 10,540


## 3.3. Решение Pandas

In [17]:
logins_extended = logins.copy()
logins_extended['id_and_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + logins['date'].dt.to_period('M').dt.to_timestamp().astype(str))
logins_extended['id_and_prev_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + (logins['date'].dt.to_period('M').dt.to_timestamp() 
       - pd.DateOffset(months=1)).astype(str))
logins_extended = logins_extended.drop_duplicates(['user_id','id_and_month'])
logins_extended

,user_id,date,id_and_month,id_and_prev_month
0,12288,2022-11-02,12288_2022-11-01,12288_2022-10-01
1,15552,2022-10-02,15552_2022-10-01,15552_2022-09-01
2,19120,2022-09-07,19120_2022-09-01,19120_2022-08-01
3,27212,2021-09-27,27212_2021-09-01,27212_2021-08-01
4,15570,2021-09-11,15570_2021-09-01,15570_2021-08-01
...,...,...,...,...
99994,21161,2021-04-25,21161_2021-04-01,21161_2021-03-01
99995,20979,2021-10-16,20979_2021-10-01,20979_2021-09-01
99996,18852,2021-12-21,18852_2021-12-01,18852_2021-11-01
99998,5541,2022-11-30,5541_2022-11-01,5541_2022-10-01


In [18]:
logins_extended['prev_month_login'] = (
    np.where(logins_extended['id_and_prev_month'].isin(logins_extended['id_and_month'])
             , 1 , 0))
logins_extended.groupby([logins_extended['date'].dt.to_period('M')])['prev_month_login'].sum()

<ipython-input-18-51ef2d549242>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logins_extended['prev_month_login'] = (


date
2021-01      0
2021-02    484
2021-03    534
2021-04    572
2021-05    544
2021-06    562
2021-07    547
2021-08    561
2021-09    553
2021-10    540
2021-11    532
2021-12    562
2022-01    541
2022-02    509
2022-03    495
2022-04    526
2022-05    510
2022-06    571
2022-07    585
2022-08    601
2022-09    566
2022-10    516
2022-11    531
Freq: M, Name: prev_month_login, dtype: int64

# 4. Задача № 4. Нарастающий итог

## 4.0. Поставнока задачи

**Контекст:** допустим, у нас есть таблица transactions в таком виде:

| date       | cash_flow |
|------------|-----------|
| 2018-01-01 | -1000     |
| 2018-01-02 | -100      |
| 2018-01-03 | 50        |
| ...        | ...       |


Где cash_flow — это выручка минус затраты за каждый день.

**Задача:** написать запрос, чтобы получить нарастающий итог для денежного потока каждый день таким образом, чтобы в конечном итоге получилась таблица в такой форме:

| date       | cumulative_cf |
|------------|---------------|
| 2018-01-01 | -1000         |
| 2018-01-02 | -1100         |
| 2018-01-03 | -1050         |
| ...        | ...           |

## 4.1. Подготовка данных

In [26]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

transactions = pd.DataFrame()
transactions['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
transactions['cash_flow'] = np.random.randint(-2000, 2000, sample_size)
transactions

,date,cash_flow
0,2021-01-01,-1021
1,2021-01-02,977
2,2021-01-03,-1666
3,2021-01-04,-178
4,2021-01-05,-1384
5,2021-01-06,1091
6,2021-01-07,-1249
7,2021-01-08,-1225
8,2021-01-09,812
9,2021-01-10,1928


In [27]:
transactions.to_sql('transactions', con, if_exists='replace',index=False)

## 4.2. Решение SQL

In [28]:
sql = '''
select
  date,
  sum(cash_flow) over (order by date) AS cumulative_cf
from transactions
group by date
order by date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,-1021
1,2021-01-02 00:00:00,-44
2,2021-01-03 00:00:00,-1710
3,2021-01-04 00:00:00,-1888
4,2021-01-05 00:00:00,-3272
5,2021-01-06 00:00:00,-2181
6,2021-01-07 00:00:00,-3430
7,2021-01-08 00:00:00,-4655
8,2021-01-09 00:00:00,-3843
9,2021-01-10 00:00:00,-1915


## 4.3. Решение SQL без оконной функции

In [29]:
sql = '''
select
  t1.date,
  sum(t2.cash_flow) as cumulative_cf
from transactions as t1
join transactions as t2 on t1.date >= t2.date
group by t1.date
order by t1.date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,-1021
1,2021-01-02 00:00:00,-44
2,2021-01-03 00:00:00,-1710
3,2021-01-04 00:00:00,-1888
4,2021-01-05 00:00:00,-3272
5,2021-01-06 00:00:00,-2181
6,2021-01-07 00:00:00,-3430
7,2021-01-08 00:00:00,-4655
8,2021-01-09 00:00:00,-3843
9,2021-01-10 00:00:00,-1915


## 4.4. Решение Pandas

In [30]:
transactions_extended = transactions.copy().sort_values('date')
transactions_extended['cumulative_cf'] = transactions_extended['cash_flow'].cumsum()
transactions_extended[['date','cumulative_cf']]

,date,cumulative_cf
0,2021-01-01,-1021
1,2021-01-02,-44
2,2021-01-03,-1710
3,2021-01-04,-1888
4,2021-01-05,-3272
5,2021-01-06,-2181
6,2021-01-07,-3430
7,2021-01-08,-4655
8,2021-01-09,-3843
9,2021-01-10,-1915


# 5. Задача № 5. Скользящее среднее

## 5.0. Постановка задачи

**Контекст:** допустим, у нас есть таблица signups в таком виде:

| date       | sign_ups |
|------------|----------|
| 2018-01-01 | 10       |
| 2018-01-02 | 20       |
| 2018-01-03 | 50       |
| ...        | ...      |
| 2018-10-01 | 35       |


**Задача:** написать запрос, чтобы получить 7-дневное скользящее среднее ежедневных регистраций

## 5.1. Подготовка данных

In [31]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

signups = pd.DataFrame()
signups['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
signups['sign_ups'] = np.random.randint(0, 100, sample_size)
signups

,date,sign_ups
0,2021-01-01,24
1,2021-01-02,3
2,2021-01-03,40
3,2021-01-04,70
4,2021-01-05,79
5,2021-01-06,73
6,2021-01-07,68
7,2021-01-08,87
8,2021-01-09,63
9,2021-01-10,58


In [32]:
signups.to_sql('signups', con, if_exists='replace',index=False)

## 5.2. Решение SQL

In [59]:
sql = '''
select
  s1.date,
  s1.sign_ups,
  avg(s2.sign_ups)
from signups as s1

join signups as s2 on s1.date < date(s2.date, '7 days') and s1.date >= s2.date
group by s1.date
'''

results = select(sql)
results

,date,sign_ups,avg(s2.sign_ups)
0,2021-01-01 00:00:00,24,24.000000
1,2021-01-02 00:00:00,3,13.500000
2,2021-01-03 00:00:00,40,22.333333
3,2021-01-04 00:00:00,70,34.250000
4,2021-01-05 00:00:00,79,43.200000
5,2021-01-06 00:00:00,73,48.166667
6,2021-01-07 00:00:00,68,51.000000
7,2021-01-08 00:00:00,87,60.000000
8,2021-01-09 00:00:00,63,68.571429
9,2021-01-10 00:00:00,58,71.142857


## 5.3. Решение Pandas

In [61]:
signups_extended = signups.copy().set_index('date')
signups_extended.rolling('7D').mean()

,sign_ups
date,
2021-01-01,24
2021-01-02,3
2021-01-03,40
2021-01-04,70
2021-01-05,79
2021-01-06,73
2021-01-07,68
2021-01-08,87
2021-01-09,63
